In [ ]:
#-----------Detección de especies endémicas en entornos controlados-----------

import numpy as np
import cv2
from PIL import Image, ImageTk
from ultralytics import YOLO
import supervision as sv
from pathlib import Path
import tkinter as tk
from tkinter import filedialog, Toplevel, Label, Button

# ---------- Cargar modelo YOLO ----------
yolo = YOLO(r"C:/Users/Andre/runs/detect/yolov10_Entrenado/weights/best.pt")
yolo.verbose = False

# ---------- Función para mejora de imagen (CLAHE + sharpen) ----------
def mejorar_imagen(frame_bgr):
    lab = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l_enhanced = clahe.apply(l)
    lab_enhanced = cv2.merge((l_enhanced, a, b))
    enhanced_bgr = cv2.cvtColor(lab_enhanced, cv2.COLOR_LAB2BGR)
    sharpen_kernel = np.array([[-1, -1, -1],
                               [-1, 9, -1],
                               [-1, -1, -1]])
    return cv2.filter2D(enhanced_bgr, -1, sharpen_kernel)

# ---------- Función para detección desde archivo ----------
def detectar_desde_imagen():
    file_path = filedialog.askopenfilename(
        title="Selecciona una imagen",
        filetypes=[("Imágenes", "*.jpg *.jpeg *.png *.bmp")]
    )
    if not file_path:
        print("❌ Imagen no seleccionada.")
        return

    img = Image.open(file_path).convert("RGB")
    image_np = np.array(img)

    res = yolo(image_np, conf=0.15)[0]
    boxes = res.boxes.xyxy.cpu().numpy()
    confidences = res.boxes.conf.cpu().numpy()
    class_ids = res.boxes.cls.cpu().numpy().astype(int)

    print(f"📦 Detecciones encontradas: {len(boxes)}")

    detections = sv.Detections(
        xyxy=boxes,
        confidence=confidences,
        class_id=class_ids
    )
    labels = [f"{yolo.names[c]} {conf:.2f}" for c, conf in zip(class_ids, confidences)]

    box_annotator = sv.BoxAnnotator(thickness=2)
    label_annotator = sv.LabelAnnotator(text_scale=0.4)

    annotated = box_annotator.annotate(scene=image_np.copy(), detections=detections)
    annotated = label_annotator.annotate(scene=annotated, detections=detections, labels=labels)

    image_resized = cv2.resize(annotated, (700, 600))
    cv2.imshow("🦎 Especies Detectadas", image_resized)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# ---------- Función para cámara en vivo ----------
def abrir_ventana_camara():
    ventana_camara = Toplevel(root)
    ventana_camara.title("📷 Cámara en Vivo - Tomar Foto y Detectar")
    ventana_camara.geometry("800x600")

    label_img = Label(ventana_camara)
    label_img.pack()

    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

    def mostrar_video():
        ret, frame = cap.read()
        if ret:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            imgtk = ImageTk.PhotoImage(image=Image.fromarray(frame_rgb))
            label_img.imgtk = imgtk
            label_img.configure(image=imgtk)
        ventana_camara.after(10, mostrar_video)

    def capturar_y_detectar():
        ret, frame = cap.read()
        if not ret:
            print("❌ Error al capturar imagen.")
            return
        mejorado = mejorar_imagen(frame)
        img_rgb = cv2.cvtColor(mejorado, cv2.COLOR_BGR2RGB)
        res = yolo(img_rgb, conf=0.10)[0]

        boxes = res.boxes.xyxy.cpu().numpy()
        confidences = res.boxes.conf.cpu().numpy()
        class_ids = res.boxes.cls.cpu().numpy().astype(int)

        if len(boxes) == 0:
            print("⚠️ No se detectaron especies.")
        else:
            print("✅ Especies detectadas:")
            for cls, conf in zip(class_ids, confidences):
                print(f"- {yolo.names[cls]} ({conf:.2f})")

        detections = sv.Detections(
            xyxy=boxes,
            confidence=confidences,
            class_id=class_ids
        )
        labels = [f"{yolo.names[c]} {conf:.2f}" for c, conf in zip(class_ids, confidences)]

        box_annotator = sv.BoxAnnotator(thickness=2)
        label_annotator = sv.LabelAnnotator(text_scale=0.7)

        frame_annotated = box_annotator.annotate(scene=mejorado.copy(), detections=detections)
        frame_annotated = label_annotator.annotate(scene=frame_annotated, detections=detections, labels=labels)

        cv2.imshow("📸 Resultado de Detección", frame_annotated)

    boton_capturar = Button(ventana_camara, text="📸 Capturar y Detectar", font=("Arial", 13), command=capturar_y_detectar)
    boton_capturar.pack(pady=10)

    def cerrar_camara():
        cap.release()
        cv2.destroyAllWindows()
        ventana_camara.destroy()

    ventana_camara.protocol("WM_DELETE_WINDOW", cerrar_camara)
    mostrar_video()

# ---------- Ventana Principal ----------
root = tk.Tk()
root.title("🦎 Detección de Especies Endémicas")
root.geometry("400x250")
root.resizable(False, False)

titulo = Label(root, text="Menú de Detección", font=("Arial", 16))
titulo.pack(pady=20)

btn_cargar = Button(root, text="📁 Cargar Imagen y Detectar", font=("Arial", 13), command=detectar_desde_imagen)
btn_cargar.pack(pady=10)

btn_camara = Button(root, text="📷 Tomar Foto y Detectar", font=("Arial", 13), command=abrir_ventana_camara)
btn_camara.pack(pady=10)

root.mainloop()
